In [17]:
cd_sd = camels_data.summary_data.loc[:,~camels_data.summary_data.columns.duplicated()]

In [18]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define the list of cities and their latitudes/longitudes
cities = cd_sd['station_name']
lats = cd_sd['lat_outlet']
longs = cd_sd['long_outlet']

# Generate a random priority for each city between 1 and 5
priority = np.random.randint(1, 6, size=len(cities))
state= cd_sd['state_outlet']

# Create the DataFrame with the city data
data = {'cityname': cities,
        'lats': lats,
        'longs': longs,
        'States': state,
        'priority': priority
        }

df = pd.DataFrame(data)

In [19]:
state_mapping = {'QLD': 1, 'NSW': 2, 'SA': 3, 'VIC': 4, 'ACT': 5, 'WA': 6, 'NT': 7, 'TAS': 8}
df['state_num'] = df['States'].map(state_mapping)

In [ ]:
# Load the shapefile of Australia
australia = gpd.read_file('STE_2021_AUST_SHP_GDA2020/STE_2021_AUST_GDA2020.shp')

# Define the CRS of the shapefile manually
australia.crs = 'epsg:7844'

# Create a GeoDataFrame from the DataFrame of cities
gdf_cities = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longs, df.lats))

# Set the CRS of the GeoDataFrame to EPSG 7844
# https://epsg.io/7844
gdf_cities.crs = 'epsg:7844'

# Reproject the GeoDataFrame of cities to match the CRS of the shapefile
gdf_cities = gdf_cities.to_crs(australia.crs)

# Perform a spatial join to link the cities to their corresponding polygons in the shapefile
gdf_cities = gpd.sjoin(gdf_cities, australia, predicate='within')

# Set up the plot
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(8, 8))

# # Define a custom dark color palette
custom_palette = sns.color_palette(['darkblue', 'black', 'purple','darkred', 'darkgreen', 'darkorange', 'brown' , 'blue'], n_colors=len(df['state_num'].unique()))

# Plot the cities colored by priority with adjustments
sns.scatterplot(ax=ax, data=gdf_cities, x='longs', y='lats', hue='States', s=15, palette=custom_palette, edgecolor='black', alpha=0.8, legend='full', zorder=2)


# Set x-axis limits
ax.set_xlim(110, 160)

# Add the shapefile of Australia as a background map
australia.plot(ax=ax, color='lightgrey', edgecolor='white', zorder=1)

# Set the plot title and axis labels
plt.title('Catchments across Australia')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()